**Connect to database**

In [3]:
from pymongo import MongoClient

# Connect to the MongoDB server
client = MongoClient('mongodb://tower.grove:27017/')

client.list_database_names()

['240115-600k', 'admin', 'config', 'local']

In [4]:
# Choose database and collection
db = client['240115-600k']
collection = db['rd0']

collection.count_documents({})

2505128

In [13]:
# Count documents with no SRVFAIL status
nosrvfail = collection.count_documents({'responses': {'$not': {'$elemMatch': {'status': 'SRVFAIL'}}}})
print(nosrvfail)

1942391


**Count eligible for RD0 decision, how many respect RD0**

In [17]:
ns_agg = db['ns_aggregated']
ns_agg.create_index([('ns', 1), ('vp', 1)], unique=True)

'ns_1_vp_1'

In [18]:
from lib.basic_measurement import BasicMeasurement

nosrvfail = collection.find({'responses': {'$not': {'$elemMatch': {'status': 'SRVFAIL'}}}})

satisfy_condition = 0
respect_rd0 = 0

for doc in nosrvfail:
  m = BasicMeasurement(doc)
  eligible = m.cs_respects_rd_zero()
  satisfy_condition += 1 if eligible else 0
  if eligible:
    respect_rd0 += 1 if m.respects_rd_zero() else 0
    agg = {
      'ns': m.get_nameserver(),
      'vp': m.get_vantagepoint(),
      'rd0': m.respects_rd_zero()
    }
    ns_agg.insert_one(agg)

print(satisfy_condition)
print(respect_rd0)

# runtime 284 minutes

# 1. Count documents with no SRVFAIL status
# 2. Iterate over these documents
# 3. For each document, create a BasicMeasurement object


1488853
559890


In [20]:
ns_agg.count_documents({'rd0': True})

559890

**Example db query, no projections yet**

In [14]:
num_logentries_gte = lambda x: {'$expr': {'$gte': [{'$size': '$logentries'}, x]}}
random_subdomains = lambda x: {'queries': {'$elemMatch': {'random_subdomains': x}}}
at_least_num_status = lambda s, num: {'$expr': 
                                 {'$gte': [
                                   {'$size': 
                                    {'$filter': 
                                     {'input': '$responses', 'as': 'r', 'cond': {'$eq': ['$$r.status', s]}}}
                                    },
                                  num]}
                                }

In [15]:

collection.count_documents(num_logentries_gte(1))

collection.count_documents(at_least_num_status('NOERROR', 3))

2108665